# Creating SPICE Kernels for Various Sites

## This shows how to create the SPICE kernel for the HCRO and VA7MM sites.

VA7MM is the transmitter near Vancouver.

Note: The methodology used her was derived from some useful code by Stu Pilorz.

This is in 2 steps:

  1. Create a definitions file for the sites. This requires you to convert the lat/lon of the sites to rectangular coordinates. Then create a text definitions file.
  2. Us the NAIF utility program pinpoint to read the definitions file just created and output a SPIKCE kernel.

In [1]:
import os
import numpy as np
import spiceypy

In [2]:
# furnsh loads a kernel for our use.
# "pck00010.tpc" makes available for use in SPICE-based application
# software orientation and size/shape data for natural bodies.
# This is how spiceypy knows about Earth locations.
spiceypy.furnsh('./kernels/pck00010.tpc')

In [6]:
# Define the locations for our sites
HCRO_LAT = 40.8172
HCRO_LON = -121.4698
VA7MM_LAT = 40.8172
VA7MM_LON = -122.7611

In [9]:
# Use spiceypy.srfrec to convert the lat/lon of the locations to
# planetocentric latitude and longitude of a surface
# point on a specified body to rectangular coordinates.
# See https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/FORTRAN/spicelib/srfrec.html
hcro_xyz  = spiceypy.srfrec(399, np.radians(HCRO_LON),  np.radians(HCRO_LAT))
va7mm_xyz  = spiceypy.srfrec(399, np.radians(VA7MM_LON),  np.radians(VA7MM_LAT))
print(hcro_xyz)
print(va7mm_xyz)

[-2516.29320604 -4111.07969591  4163.06565411]
[-2608.29952976 -4053.32965942  4163.06565411]


### Create a site definitions file that looks like this (insert the XYZ coordinates you just calculated)

   \begindata

     SITES     = ( 'VA7MM',
                   'HCRO' )

     VA7MM_FRAME  = 'ITRF93'
     VA7MM_IDCODE = 399501
     VA7MM_XYZ    = (-2246.79898141, -3491.55334585,  4825.41698992)
     VA7MM_CENTER = 399
     VA7MM_UP     = 'Z'
     VA7MM_NORTH  = 'X'
     VA7MM_BOUNDS = ( @2000-JAN-1, @2030-JAN-1)

     HCRO_FRAME   = 'ITRF93'
     HCRO_IDCODE  = 399502
     HCRO_XYZ     = (-2516.29320604, -4111.07969591,  4163.06565411)
     HCRO_CENTER  = 399
     HCRO_UP      = 'Z'
     HCRO_NORTH   = 'X'
     HCRO_BOUNDS  = ( @2000-JAN-1, @2030-JAN-1)

   \begintext

In [15]:
# Create the text, insert the XYZ locations.

text = """\\begindata

 SITES     = ( 'VA7MM',
               'HCRO' )

 VA7MM_FRAME  = 'ITRF93'
 VA7MM_IDCODE = 399501
 VA7MM_XYZ    = (-2246.79898141, -3491.55334585,  4825.41698992)
 VA7MM_CENTER = 399
 VA7MM_UP     = 'Z'
 VA7MM_NORTH  = 'X'
 VA7MM_BOUNDS = ( @2000-JAN-1, @2030-JAN-1)

 HCRO_FRAME   = 'ITRF93'
 HCRO_IDCODE  = 399502
 HCRO_XYZ     = (-2516.29320604, -4111.07969591,  4163.06565411)
 HCRO_CENTER  = 399
 HCRO_UP      = 'Z'
 HCRO_NORTH   = 'X'
 HCRO_BOUNDS  = ( @2000-JAN-1, @2030-JAN-1)
 
\\begintext"""

In [20]:
# Write the text to the file. 
# Create the direcory if it does not exist
CUSTOM_SITES_DIRNAME = "custom_site_defs"
os.makedirs(CUSTOM_SITES_DIRNAME, exist_ok=True)
f = open(f"{CUSTOM_SITES_DIRNAME}{os.sep}hcro_and_va7mm.defs", "w")
f.write(text)
f.close()

### Now we have a definition file for these sites. Create the kernel.
Uses the defs file we just created.
Use the pinpoint utility to create the kernel.
\
On my MAC I downloaded pinpoint from https://naif.jpl.nasa.gov/naif/utilities.html and moved it to /usr/local/bin, had to chmod to make it executable.

In [6]:
CUSTOM_SITES_DIRNAME = "custom_site_defs"
CUSTOM_SITE_DEFS_FILENAME = "hcro_and_va7mm.defs"
KERNELS_DIRNAME = "kernels"
KERNEL_FILENAME = "hcro_and_va7mm"

command =  f"pinpoint -def {CUSTOM_SITES_DIRNAME}{os.sep}{CUSTOM_SITE_DEFS_FILENAME}"
command += " -pck ./kernels/pck00010.tpc"
command += f" -spk {KERNELS_DIRNAME}{os.sep}{KERNEL_FILENAME}.bsp"
command += f" -fk {KERNELS_DIRNAME}{os.sep}{KERNEL_FILENAME}.tf"
print(command)
os.system(command)

pinpoint -def custom_site_defs/hcro_and_va7mm.defs -pck ./kernels/pck00010.tpc -spk kernels/hcro_and_va7mm.bsp -fk kernels/hcro_and_va7mm.tf


0

# hcro_and_va7mm.tf and hcro_and_va7mm.bsp should exist in the kernels directory.

You have done it!